In [1]:
import pandas as pd
import sqlite3

In [2]:
# Read the CSV file into a pandas DataFrame
data = pd.read_csv('E:\Training\Mentorness\Corona Virus Analysis in SQL\Corona Virus Dataset.csv')

# Create an in-memory SQLite database
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Create a table in the database and insert the data from the DataFrame
data.to_sql('corona_data', conn, index=False)

78386

In [3]:
# Function to execute SQL queries and return results as DataFrame
def run_query(query):
    return pd.read_sql(query, conn)

In [4]:
# Q1. Check NULL values for all columns
query_1 = """
    SELECT *
    FROM corona_data
    WHERE 
        Province IS NULL OR
        "Country/Region" IS NULL OR
        Latitude IS NULL OR 
        Longitude IS NULL OR
        Date IS NULL OR
        Confirmed IS NULL OR
        Deaths IS NULL OR
        Recovered IS NULL;  
"""
null_values = run_query(query_1)
print("Null Values:\n", null_values)

Null Values:
 Empty DataFrame
Columns: [Province, Country/Region, Latitude, Longitude, Date, Confirmed, Deaths, Recovered]
Index: []


In [5]:
# Q2. Update NULL values with zeros for all columns
query_2 = """
    UPDATE corona_data
    SET 
        Province = COALESCE(Province, ''),
        "Country/Region" = COALESCE("Country/Region", ''),
        Latitude = COALESCE(Latitude, 0),
        Longitude = COALESCE(Longitude, 0),
        Date = COALESCE(Date, ''),
        Confirmed = COALESCE(Confirmed, 0),
        Deaths = COALESCE(Deaths, 0),
        Recovered = COALESCE(Recovered, 0);
"""
conn.execute(query_2)
conn.commit()

In [7]:
# Q3. check total number of rows
query_3 = """
    SELECT count(*) AS Total_Number_Of_Rows
    FROM corona_data;
"""
total_rows = run_query(query_3)
print(total_rows)

   Total_Number_Of_Rows
0                 78386


In [8]:
query_4 = "SELECT MIN(Date) AS Start_Date, MAX(Date) AS End_Date FROM corona_data;"
dates = run_query(query_4)
print(dates)

   Start_Date    End_Date
0  01-01-2021  31-12-2020


In [9]:
# Q5. Number of months present in dataset
query_5 = """
    SELECT COUNT(DISTINCT strftime('%m-%Y', substr(Date, 7, 4) || '-' || substr(Date, 4, 2) || '-' || substr(Date, 1, 2))) AS num_months
    FROM corona_data;
"""
num_months_result = run_query(query_5)
print("Number of Months Present in Dataset:", num_months_result['num_months'].values[0])

Number of Months Present in Dataset: 18


In [14]:
# Q6. Find average for confirmed, deaths, recovered in each month in Date column
query_6 = """
    SELECT 
        strftime('%Y-%m', date) AS month,
        AVG(confirmed) AS avg_confirmed,
        AVG(deaths) AS avg_deaths,
        AVG(recovered) AS avg_recovered
    FROM corona_data
    GROUP BY month
    ORDER BY month;
"""


result_6 = run_query(query_6)
print("Average for Confirmed, Deaths, Recovered in Each Month:")
print(result_6)

Average for Confirmed, Deaths, Recovered in Each Month:
  month  avg_confirmed  avg_deaths  avg_recovered
0  None    2156.828311    46.53757    1442.726354


In [25]:
# Q7. Find most frequent value for confirmed, deaths, recovered each month
# Find most frequent value for confirmed cases
query_confirmed = """
    SELECT confirmed AS Most_Frequent_Confirmed
    FROM corona_data
    GROUP BY confirmed
    ORDER BY COUNT(*) DESC
    LIMIT 1;
"""
result_confirmed = run_query(query_confirmed)

# Find most frequent value for deaths
query_deaths = """
    SELECT deaths AS Most_Frequent_Deaths
    FROM corona_data
    GROUP BY deaths
    ORDER BY COUNT(*) DESC
    LIMIT 1;
"""
result_deaths = run_query(query_deaths)

# Find most frequent value for recovered cases
query_recovered = """
    SELECT recovered AS Most_Frequent_Recovered
    FROM corona_data
    GROUP BY recovered
    ORDER BY COUNT(*) DESC
    LIMIT 1;
"""
result_recovered = run_query(query_recovered)

# Display results
print("Most Frequent Value for Confirmed Cases:", result_confirmed)
print("Most Frequent Value for Deaths:", result_deaths)
print("Most Frequent Value for Recovered Cases:", result_recovered)

Most Frequent Value for Confirmed Cases:    Most_Frequent_Confirmed
0                        0
Most Frequent Value for Deaths:    Most_Frequent_Deaths
0                     0
Most Frequent Value for Recovered Cases:    Most_Frequent_Recovered
0                        0


In [6]:
# Q8: Find minimum values for confirmed, deaths, recovered per year
query_8 = """
    SELECT 
        strftime('%Y', date) AS year,
        MIN(confirmed) AS min_confirmed,
        MIN(deaths) AS min_deaths,
        MIN(recovered) AS min_recovered
    FROM corona_data
    GROUP BY strftime('%Y', date);
"""
result_8 = run_query(query_8)
print("Minimum Values of Confirmed, Deaths, Recovered Per Year:")
print(result_8)

Minimum Values of Confirmed, Deaths, Recovered Per Year:
   year  min_confirmed  min_deaths  min_recovered
0  None              0           0              0


In [26]:
# Q9. Find maximum values of confirmed, deaths, recovered per year
query_9 = """
    SELECT 
        strftime('%Y', date) AS year,
        MAX(confirmed) AS max_confirmed,
        MAX(deaths) AS max_deaths,
        MAX(recovered) AS max_recovered
    FROM corona_data
    GROUP BY strftime('%Y', date);
"""
result_9 = run_query(query_9)
print("Maximum Values of Confirmed, Deaths, Recovered Per Year:")
print(result_9)

Maximum Values of Confirmed, Deaths, Recovered Per Year:
   year  max_confirmed  max_deaths  max_recovered
0  None         823225        7374        1123456


In [27]:
# Q10. The total number of case of confirmed, deaths, recovered each month
query_10 = """
    SELECT 
        strftime('%Y-%m', date) AS month,
        SUM(confirmed) AS total_confirmed,
        SUM(deaths) AS total_deaths,
        SUM(recovered) AS total_recovered
    FROM corona_data
    GROUP BY strftime('%Y-%m', date);
"""
result_10 = run_query(query_10)
print("Total Number of Cases of Confirmed, Deaths, Recovered Each Month:")
print(result_10)

Total Number of Cases of Confirmed, Deaths, Recovered Each Month:
  month  total_confirmed  total_deaths  total_recovered
0  None        169065144       3647894        113089548


In [36]:
# Q11. Check how corona virus spread out with respect to confirmed case
query_11 = """
    SELECT 
        COUNT(*) AS total_cases,
        AVG(confirmed) AS avg_confirmed,
        SUM((confirmed - (SELECT AVG(confirmed) FROM corona_data)) * (confirmed - (SELECT AVG(confirmed) FROM corona_data))) / COUNT(*) AS variance_confirmed
    FROM corona_data;
"""

result_11 = run_query(query_11)

# Calculate standard deviation in Python
variance_confirmed = result_11['variance_confirmed'].values[0]
stdev_confirmed = variance_confirmed ** 0.5

# Add standard deviation to the result DataFrame
result_11['stdev_confirmed'] = stdev_confirmed

print("Spread of Coronavirus with Respect to Confirmed Cases:")
print(result_11)

Spread of Coronavirus with Respect to Confirmed Cases:
   total_cases  avg_confirmed  variance_confirmed  stdev_confirmed
0        78386    2156.828311        1.572889e+08     12541.488152


In [38]:
# Q12. Check how corona virus spread out with respect to death case per month
query_12 = """
    SELECT 
        strftime('%Y-%m', Date) AS month,
        COUNT(*) AS total_deaths,
        AVG(deaths) AS avg_deaths,
        SUM((deaths - (SELECT AVG(deaths) FROM corona_data)) * (deaths - (SELECT AVG(deaths) FROM corona_data))) / COUNT(*) AS variance_deaths
    FROM corona_data
    GROUP BY month
    ORDER BY month;
"""

result_12 = run_query(query_12)

# Calculate standard deviation in Python
variance_deaths = result_12['variance_deaths']
stdev_deaths = variance_deaths.apply(lambda x: x ** 0.5)

# Add standard deviation to the result DataFrame
result_12['stdev_deaths'] = stdev_deaths

print("Spread of Coronavirus with Respect to Death Cases:")
print(result_12)

Spread of Coronavirus with Respect to Death Cases:
  month  total_deaths  avg_deaths  variance_deaths  stdev_deaths
0  None         78386    46.53757     45892.018854    214.224226


In [39]:
# Q13. Check how corona virus spread out with respect to recovered case
query_13 = """
    SELECT 
        strftime('%Y-%m', Date) AS month,
        COUNT(*) AS total_recovered,
        AVG(recovered) AS avg_recovered,
        SUM((recovered - (SELECT AVG(recovered) FROM corona_data)) * (recovered - (SELECT AVG(recovered) FROM corona_data))) / COUNT(*) AS variance_recovered
    FROM corona_data
    GROUP BY month
    ORDER BY month;
"""

result_13 = run_query(query_13)

# Calculate standard deviation in Python
variance_recovered = result_13['variance_recovered']
stdev_recovered = variance_recovered.apply(lambda x: x ** 0.5)

# Add standard deviation to the result DataFrame
result_13['stdev_recovered'] = stdev_recovered

print("Spread of Coronavirus with Respect to Recovered Cases:")
print(result_13)

Spread of Coronavirus with Respect to Recovered Cases:
  month  total_recovered  avg_recovered  variance_recovered  stdev_recovered
0  None            78386    1442.726354        1.070295e+08     10345.507395


In [43]:
# Q14. Find Country having highest number of the Confirmed case
query_14 = """
    SELECT "Country/Region", MAX(Confirmed) AS max_confirmed
    FROM corona_data
    GROUP BY "Country/Region"
    ORDER BY max_confirmed DESC
    LIMIT 1;
"""

result_14 = run_query(query_14)
print("Country with the Highest Number of Confirmed Cases:")
print(result_14)

Country with the Highest Number of Confirmed Cases:
  Country/Region  max_confirmed
0         Turkey         823225


In [46]:
# Q15. Find Country having lowest number of death cases
query_15 = """
    SELECT "Country/Region", MIN(Deaths) AS min_deaths
    FROM corona_data
    GROUP BY "Country/Region"
    ORDER BY min_deaths ASC
    LIMIT 1;
"""

result_15 = run_query(query_15)
print("Country with the Lowest Number of Death Cases:")
print(result_15)

Country with the Lowest Number of Death Cases:
  Country/Region  min_deaths
0    Afghanistan           0


In [47]:
# Q16. Find top 5 countries having highest recovered cases
query_16 = """
    SELECT "Country/Region", SUM(Recovered) AS total_recovered
    FROM corona_data
    GROUP BY "Country/Region"
    ORDER BY total_recovered DESC
    LIMIT 5;
"""

result_16 = run_query(query_16)
print("Top 5 Countries with the Highest Number of Recovered Cases:")
print(result_16)

Top 5 Countries with the Highest Number of Recovered Cases:
  Country/Region  total_recovered
0          India         28089649
1         Brazil         15400169
2             US          6303715
3         Turkey          5202251
4         Russia          4745756
